In [3]:
import Functions

In [4]:
Functions.Main()

                      Scrap_weather_temperature_Script                          
ــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ
{'city': 'Cairo', 'Temperature': 'max = 21°C | min = 8°C', 'humidity': '65%', 'Air_Pressure': '', 'wind_speed': '30 Km/h', 'Date': '19 March 2022'}
{'city': 'Cairo', 'Temperature': 'max = 19°C | min = 8°C', 'humidity': '59%', 'Air_Pressure': '', 'wind_speed': '31 Km/h', 'Date': '20 March 2022'}
{'city': 'Cairo', 'Temperature': 'max = 19°C | min = 7°C', 'humidity': '63%', 'Air_Pressure': '', 'wind_speed': '16 Km/h', 'Date': '21 March 2022'}
{'city': 'Cairo', 'Temperature': 'max = 16°C | min = 8°C', 'humidity': '76%', 'Air_Pressure': '', 'wind_speed': '15 Km/h', 'Date': '22 March 2022'}
{'city': 'Cairo', 'Temperature': 'max = 16°C | min = 8°C', 'humidity': '60%', 'Air_Pressure': '', 'wind_speed': '16 Km/h', 'Date': '23 March 2022'}
